In [1]:
BLACKHOLE = True


import os
import sys
from pathlib import Path


if BLACKHOLE:
    workspace_path = os.path.expandvars('$BLACKHOLE')
    sys.path.append(workspace_path+'/DeepLearning/02456_news_project/src')
    DATAPATH = Path(workspace_path+"/DeepLearning/ebnerd_data").expanduser()
else:
    DATAPATH = Path("~/ebnerd_data").expanduser()

DATASET = "ebnerd_demo"
# DATASET = "ebnerd_small"

In [2]:
import torch

print("torch version:", torch.__version__)

# Check gpu availability


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.zeros(1).cuda())

torch version: 2.5.1+cu124
cuda
tensor([0.], device='cuda:0')


In [8]:
# Check if the model can be loaded
from nrms import NRMSModel
from hyperparameters import HyperParameters

# hparams contains the hyperparameters of the model:
# They are:
# vocab_size: int
# word_embedding_dim: int
# dropout: float


hparams = HyperParameters(
    vocab_size=100, 
    word_embedding_dim=300, 
    dropout=0.1, 
    head_num=8,
    head_dim=64,
    attention_hidden_dim=512,
    title_size=20
)

word2vec_embedding = torch.randn(hparams.vocab_size, hparams.word_embedding_dim)

model = NRMSModel(hparams=hparams, word2vec_embedding=word2vec_embedding)

print(model)





KeyboardInterrupt



TODO HERFRA OG NED

In [3]:

from utils.data_handler import NewsDataset
from utils.constants import (
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_SUBTITLE_COL,
    DEFAULT_TITLE_COL,
    DEFAULT_USER_COL,
    DEFAULT_ARTICLE_ID_COL
)

# test
dataset = NewsDataset(dataset_path=DATAPATH.joinpath(DATASET))

#print(dataset.df_history_train.head())

#print(dataset.df_behaviors_train.head())

#print(dataset.extract_history_train_data([DEFAULT_USER_COL, DEFAULT_HISTORY_ARTICLE_ID_COL]))
#print(dataset.extract_articles_data([DEFAULT_ARTICLE_ID_COL,DEFAULT_TITLE_COL,DEFAULT_SUBTITLE_COL]))
print(dataset.get_article_info_by_id("9742305"))

[{'article_id': 9742305, 'title': 'Danmark smadrer Sverige 37-31', 'subtitle': 'Angrebsspillet flød i første halvleg, da Danmarks håndboldlandshold lørdag besejrede Sverige i en testkamp', 'last_modified_time': Timestamp('2023-06-29 06:48:26'), 'premium': False, 'body': 'De danske håndboldverdensmestre blæste Sverige af banen og viste storform i første halvleg af lørdagens testkamp, men et svensk comeback efter pausen truede med at vende det hele på hovedet.\nPå hjemmebane i Herning vandt Danmark dog i sidste ende testkampen sikkert med 37-31 og sluttede dermed som etter i træningsturneringen EHF Euro Cup.\nDanskerne har indimellem haft problemer med det svenske forsvar, men det var langtfra tilfældet i første halvdel af lørdagens kamp.\nRasmus Lauge styrede den danske offensiv, som uden problemer spillede sig til den ene store chance efter den anden.\nVeszprém-spilleren var skarp, når han selv tog chancen, og hans høje niveau skabte også plads til holdkammeraterne, som han elegant fan

In [11]:
# Use the dataloader to separate the data into batches / validation / test sets
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

for batch in dataloader:
    print(batch)
    break


TypeError: object of type 'NewsDataset' has no len()

In [ ]:
# Train the model

from train import train

train(
    model=model,
    dataloader=dataloader,
    hparams=hparams,
    device=device
)

# Save the model
torch.save(model.state_dict(), "model.pth")